In [31]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tsa.stattools import adfuller

In [36]:
rmse_temp = []
rmse_hum = []
rmse_gas = []


In [37]:
#we go from minute to minute
fd =  open('../../server/config.json', 'r')
config = json.load(fd)
fd.close()
minutes = 30
for i in range(0, 30):
    query_temp = 'from(bucket:"temp")' \
            ' |> range(start:2022-06-14T12:'+str(minutes+i)+':00Z, stop:2022-06-14T15:'+str(minutes+i)+':00Z)'\
            ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
    client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()  
    result = query_api.query_data_frame(query_temp)
    df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
    df_temp = df.copy()
    df_temp = df_temp.loc[df_temp['sensor'] == '0']
    df_temp = df_temp.drop(columns=['lat', 'lon', 'sensor'])
    df_temp = df_temp.rename(columns = {'_time': 'Time', '_value': 'y'})
    df_temp['Time'] = df_temp['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
    df_temp
    l = len(df_temp.values)
    splitpoint = int(l*0.90)
    train = df_temp['y'][:splitpoint]
    test = df_temp['y'][splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test.index))
    rmse_temp.append(math.sqrt(mean_squared_error(test, predictions)))

In [38]:
for i in range(0, 30):
    query_hum = 'from(bucket:"hum")' \
            ' |> range(start:2022-06-14T12:'+str(minutes+i)+':00Z, stop:2022-06-14T15:'+str(minutes+i)+':00Z)'\
            ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
    client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()  
    result = query_api.query_data_frame(query_hum)
    df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
    df_hum = df.copy()
    df_hum = df_hum.loc[df_hum['sensor'] == '0']
    df_hum = df_hum.drop(columns=['lat', 'lon', 'sensor'])
    df_hum = df_hum.rename(columns = {'_time': 'Time', '_value': 'y'})
    df_hum['Time'] = df_hum['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
    df_hum
    l = len(df_hum.values)
    splitpoint = int(l*0.90)
    train = df_hum['y'][:splitpoint]
    test = df_hum['y'][splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test.index))
    rmse_hum.append(math.sqrt(mean_squared_error(test, predictions)))

In [39]:
for i in range(0, 30):
    query_gas = 'from(bucket:"smoke")' \
            ' |> range(start:2022-06-14T12:'+str(minutes+i)+':00Z, stop:2022-06-14T15:'+str(minutes+i)+':00Z)'\
            ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
    client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()  
    result = query_api.query_data_frame(query_gas)
    df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
    df_gas = df.copy()
    df_gas = df_gas.loc[df_gas['sensor'] == '0']
    df_gas = df_gas.drop(columns=['lat', 'lon', 'sensor'])
    df_gas = df_gas.rename(columns = {'_time': 'Time', '_value': 'y'})
    df_gas['Time'] = df_gas['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
    df_gas
    l = len(df_gas.values)
    splitpoint = int(l*0.90)
    train = df_gas['y'][:splitpoint]
    test = df_gas['y'][splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test.index))
    rmse_gas.append(math.sqrt(mean_squared_error(test, predictions)))

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [50]:
series = ['Temp', 'Hum', 'Gas']

In [59]:
df_results = pd.DataFrame.from_dict({'Temp': rmse_temp, 'Hum':rmse_hum, 'Gas':rmse_gas})
described = df_results.describe()
for s in series:
    mean = described.loc['mean'][s]
    conf = described.loc['std'][s]/math.sqrt(described.loc['count'][s])
    print('Mean for ' + s +' {:.3g}'.format(mean) + ' conf: {:.3g}'.format(conf) + ' high bound: {:.3g}'.format(mean+conf) + ' low bound: {:.3g}'.format(mean-conf))



Mean for Temp 0.0681 conf: 0.000721 high bound: 0.0689 low bound: 0.0674
Mean for Hum 0.22 conf: 0.00711 high bound: 0.227 low bound: 0.213
Mean for Gas 0.0366 conf: 0.000432 high bound: 0.037 low bound: 0.0362
